In [ ]:
# Montar Google Drive en Colab
from google.colab import drive
drive.mount('/content/drive')

!ssh-keygen -t rsa -b 4096 -C "tuemail@unir.edu" -f /content/drive/MyDrive/id_rsa -N ""
print("Copia la siguiente información para crear tu acceso SSH desde gitHub")
!cat /content/drive/MyDrive/id_rsa.pub

Mounted at /content/drive
Generating public/private rsa key pair.
/content/drive/MyDrive/id_rsa already exists.
Overwrite (y/n)? Y
Your identification has been saved in /content/drive/MyDrive/id_rsa
Your public key has been saved in /content/drive/MyDrive/id_rsa.pub
The key fingerprint is:
SHA256:dJaqkZsTLkcwKq8/RV2gDBKc5+49Nqw1eKi9ChD5wU4 tuemail@unir.edu
The key's randomart image is:
+---[RSA 4096]----+
|+.o   ..         |
| * + .  .  .     |
|o E =. .. +      |
| = +.o.o +       |
|o =.  = S        |
|.o .+o *         |
|. o+++B          |
|..+.oO..         |
|.+o=+ o          |
+----[SHA256]-----+
Copia la siguiente información para crear tu acceso SSH desde gitHub
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDJlIzwWH9+aXNnRLqXJoHKaqWul+UuRsMM11O9EyKEaefCW/R0cEt2237oSBlyEglQfnzDwoS0FvvU+swJ/Nd5h7HPicpJKxpc8DjFPahoPSnneF7rvz70DX9/AD2wz60ZQJ1xlCQL79vDVANkkpsolJ19+Ul1He8HNg2LxLq11bH13UV5BpF2jZp3nlP++WANNGZPDnP7WnNx9bsGT4qrS7W+VZLjZlFEphrB+xgG5n5DErsA9/47jDtcsB0p/RhdEGReqk49vejcVPb8QpLBuLvsaa

In [ ]:
import subprocess
import os
# Create .ssh directory if it doesn't exist
os.makedirs(os.path.expanduser('~/.ssh'), exist_ok=True)

# Create known_hosts file if it doesn't exist
known_hosts_path = os.path.expanduser('~/.ssh/known_hosts')
if not os.path.exists(known_hosts_path):
    open(known_hosts_path, 'a').close()  # Create an empty known_hosts file

!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# Step 1: Start the SSH agent
def start_ssh_agent():
    # Start the SSH agent
    process = subprocess.Popen(['ssh-agent'], stdout=subprocess.PIPE)
    output, _ = process.communicate()

    # Set environment variables for the current session
    for line in output.decode().strip().split('\n'):
        if 'SSH_AUTH_SOCK' in line or 'SSH_AGENT_PID' in line:
            key, value = line.split(';')[0].strip().split('=')
            os.environ[key] = value

# Step 2: Add your private key from Google Drive
def add_private_key(key_path):
    # Add the private key to the SSH agent
    result = subprocess.run(['ssh-add', key_path], capture_output=True, text=True)
    if result.returncode == 0:
        print("SSH key added successfully.")
    else:
        print("Error adding SSH key:")
        print(result.stderr)

# Execute the functions
start_ssh_agent()

# Path to your private key in Google Drive
private_key_path = '/content/drive/MyDrive/id_rsa'  # Adjust this path as necessary

# Add the private key to the agent
add_private_key(private_key_path)

# Optional: Test SSH connection to GitHub
print("Testing SSH connection to GitHub...")
test_connection = subprocess.run(['ssh', '-T', 'git@github.com'], capture_output=True, text=True)
print(test_connection.stdout if test_connection.returncode == 0 else test_connection.stderr)